In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from IPython.display import display

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from random import randint
from time import sleep

# Get Spotify stuff

In [33]:
# lab: dataframe with artist, title, feature

In [138]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=123456789,
                                                           client_secret=123456789))

In [153]:
# Playlists
hits100 = "3xMQTDLOIGvj3lWH5e5x6F"
hits_d = "50WijGE2KSlSS9Sn4uIZjB"
longest_pl ="6Y7JlYRTJKpSm2gNzijBvn"



In [154]:
#code to pass the 100 limit for the tracks got from the api

def get_all_tracks(playlist):
    results = sp.user_playlist_tracks("spotify", playlist)
    tracks = results['items'] # to pass in

    for oset in range(100,results['total'],100):
        results = sp.user_playlist_tracks("spotify", playlist, offset=oset)
        tracks += results['items']
    
    return tracks


all_tracks_hits = get_all_tracks(hits100)
all_tracks_hd = get_all_tracks(hits_d)
all_tracks_lpl = get_all_tracks(longest_pl)

In [155]:
# Getting all the artists

def get_artists_from_playlist(playlist_id, tracks):
    
    tracks_from_playlist = tracks
    
    artists = []
    
    for track in tracks_from_playlist:
        artist_info = track['track']['artists'][0]
        artists.append(artist_info['name'])
    
    return artists

all_artist_hits = get_artists_from_playlist(hits100,all_tracks_hits)
all_artist_hd = get_artists_from_playlist(hits_d,all_tracks_hd)
all_artist_lpl = get_artists_from_playlist(longest_pl,all_tracks_lpl)

In [156]:
#get song names & track ids

def get_song_names(tracks):
    song_actual_names = [track['track']['name'] for track in tracks]
    track_ids = [track['track']['id'] for track in tracks]
    
    return song_actual_names,track_ids

song_names_hits,id_track_hits = get_song_names(all_tracks_hits)
song_names_hd,id_track_hd = get_song_names(all_tracks_hd)
song_names_lpl,id_track_lpl = get_song_names(all_tracks_lpl)

In [157]:
#gathering features and turning them into a dataframe

def main_features_to_df(track_ids):
    the_list = []
    for i in track_ids:
        the_list.append(sp.audio_features(i))
    #looping inside the nested list to be able to convert into a df    
    converted = [i for elem in the_list for i in elem]    
    df = pd.json_normalize(converted)
    
    return df
        
df_hits = main_features_to_df(id_track_hits)
df_hd = main_features_to_df(id_track_hd)
df_lpl = main_features_to_df(id_track_lpl)

In [158]:
# adding previous single gathered features to the current df

def add_features_to_df(song_actual_names,artist_list,features_df):
    df_song_name = pd.DataFrame(song_actual_names, columns = ["song_name"])
    df_artists = pd.DataFrame(artist_list,columns = ["artists"])
    
    final_df = pd.concat([features_df, df_song_name,df_artists], axis=1)
    
    return final_df

final_df_hits = add_features_to_df(song_names_hits,all_artist_hits,df_hits)
final_df_hd = add_features_to_df(song_names_hd,all_artist_hd,df_hd)
final_df_lpl = add_features_to_df(song_names_lpl,all_artist_lpl,df_lpl)

In [159]:
final_df = pd.concat([final_df_hits, final_df_hd, final_df_lpl], ignore_index=True)

In [160]:
final_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,song_name,artists
0,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4,As It Was,Harry Styles
1,0.905,0.563,8,-6.135,1,0.1020,0.02540,0.000010,0.1130,0.324,106.998,audio_features,1rDQ4oMwGJI7B4tovsBOxc,spotify:track:1rDQ4oMwGJI7B4tovsBOxc,https://api.spotify.com/v1/tracks/1rDQ4oMwGJI7...,https://api.spotify.com/v1/audio-analysis/1rDQ...,173948,4,First Class,Jack Harlow
2,0.761,0.525,11,-6.900,1,0.0944,0.44000,0.000007,0.0921,0.531,80.870,audio_features,02MWAaffLxlfxAUY7c5dvx,spotify:track:02MWAaffLxlfxAUY7c5dvx,https://api.spotify.com/v1/tracks/02MWAaffLxlf...,https://api.spotify.com/v1/audio-analysis/02MW...,238805,4,Heat Waves,Glass Animals
3,0.922,0.451,9,-6.912,1,0.0612,0.32400,0.010500,0.0496,0.649,126.987,audio_features,36O90PhZmnMPk4FHDe2bPz,spotify:track:36O90PhZmnMPk4FHDe2bPz,https://api.spotify.com/v1/tracks/36O90PhZmnMP...,https://api.spotify.com/v1/audio-analysis/36O9...,190846,4,Take Me Home,MOMO Soundz
4,0.795,0.800,1,-6.320,1,0.0309,0.03540,0.000073,0.0915,0.934,116.032,audio_features,6JIC3hbC28JZKZ8AlAqX8h,spotify:track:6JIC3hbC28JZKZ8AlAqX8h,https://api.spotify.com/v1/tracks/6JIC3hbC28JZ...,https://api.spotify.com/v1/audio-analysis/6JIC...,202735,4,Cold Heart - PNAU Remix,Elton John
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,0.844,0.721,1,-4.667,0,0.1500,0.00233,0.000011,0.0698,0.388,94.000,audio_features,2Ngn0Gs7308cfRst4FFtAb,spotify:track:2Ngn0Gs7308cfRst4FFtAb,https://api.spotify.com/v1/tracks/2Ngn0Gs7308c...,https://api.spotify.com/v1/audio-analysis/2Ngn...,205920,4,Best Friend (feat. Doja Cat & Katja Krasavice)...,Saweetie
424,0.934,0.427,7,-5.596,0,0.0890,0.06890,0.000002,0.0820,0.609,99.978,audio_features,3sZOiHgkXpDcilUBtHSixK,spotify:track:3sZOiHgkXpDcilUBtHSixK,https://api.spotify.com/v1/tracks/3sZOiHgkXpDc...,https://api.spotify.com/v1/audio-analysis/3sZO...,140839,4,Raindrops,Katja Krasavice
425,0.618,0.480,9,-10.135,0,0.0353,0.51000,0.000059,0.1110,0.386,144.970,audio_features,2UD1csihq8aPrpM69C2XrW,spotify:track:2UD1csihq8aPrpM69C2XrW,https://api.spotify.com/v1/tracks/2UD1csihq8aP...,https://api.spotify.com/v1/audio-analysis/2UD1...,164062,4,HIGHWAY (feat. ELIF),Katja Krasavice
426,0.736,0.704,3,-7.409,0,0.0615,0.02030,0.000000,0.0501,0.894,149.995,audio_features,27NovPIUIRrOZoCHxABJwK,spotify:track:27NovPIUIRrOZoCHxABJwK,https://api.spotify.com/v1/tracks/27NovPIUIRrO...,https://api.spotify.com/v1/audio-analysis/27No...,212000,4,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X


In [161]:
final_df.to_csv(r'C:\Users\User\IH-Labs\lab-api-wrappers\spotify_list.csv', index=False)